# Laboratorio Práctico: De Prompt Simple a Prompt Avanzado — Resuelto

**Archivo:** `Laboratorio_Prompt_Engineering_Resuelto.ipynb`

Este notebook contiene las soluciones completas a los ejercicios de los archivos `EJERCICIOS.md` y `EJERCICIOSPARTE2.md`. Cada ejercicio incluye:

- El *prompt* (simple o avanzado).
- La respuesta simulada del modelo (tal como si la hubieras pegado).
- Código ejecutable cuando aplica.
- Reflexiones comparativas.

---


## Parte 1 — EJERCICIOS.md

### Ejercicio 1: Generación de una Función de Código

**Prompt Simple:**
```
Haz una función en Python para validar una contraseña.
```

**Prompt Avanzado:** (resumen)
- Rol: desarrollador senior en ciberseguridad
- Reglas: 12+ caracteres, mayúscula, minúscula, número, carácter especial
- Salida: función `validar_contrasena` con docstrings y 3 ejemplos


In [ ]:
def validar_contrasena(contrasena: str) -> bool:
    """Valida si una contraseña cumple con los criterios de seguridad.
    
    Reglas:
    - Longitud mínima: 12 caracteres.
    - Al menos una letra mayúscula.
    - Al menos una letra minúscula.
    - Al menos un dígito.
    - Al menos un carácter especial (por ejemplo: @# $%&!).
    
    Parámetros
    ----------
    contrasena : str
        La contraseña a validar.
    
    Retorna
    -------
    bool
        True si la contraseña cumple todas las reglas, False en caso contrario.
    """
    if not isinstance(contrasena, str):
        return False
    if len(contrasena) < 12:
        return False
    has_upper = any(c.isupper() for c in contrasena)
    has_lower = any(c.islower() for c in contrasena)
    has_digit = any(c.isdigit() for c in contrasena)
    special_chars = set("@#$%&!^*()-_=+[]{};:,.<>?/|\\~`")
    has_special = any(c in special_chars for c in contrasena)
    return all([has_upper, has_lower, has_digit, has_special])

# Ejemplos de uso
ejemplos = {
    "valida": "Str0ngP@ssw0rd!",
    "invalida_corta": "Shrt1!",
    "invalida_sin_mayus": "str0ngp@ssw0rd!"
}

for nombre, pwd in ejemplos.items():
    print(f"{nombre}: {pwd} -> {validar_contrasena(pwd)}")

**Respuesta simulada del modelo (prompt avanzado):**

```python
# Código: función validar_contrasena (se muestra arriba)
```

**Ejemplos de uso y salida esperada:**
- `Str0ngP@ssw0rd!` → `True`
- `Shrt1!` → `False` (demasiado corta)
- `str0ngp@ssw0rd!` → `False` (sin mayúscula)

**Reflexión comparativa:**
- El prompt simple podría generar una función más corta sin docstrings ni ejemplos.
- El prompt avanzado produce una función lista para integrarse, con docstrings y ejemplos claros.

---


### Ejercicio 2: Extracción de Información a JSON

**Texto de entrada:**

> "La reunión de lanzamiento del Proyecto Fénix será el 15 de octubre de 2024 a las 10:30 AM en la Sala Júpiter. Los asistentes confirmados son Ana García (Líder de Proyecto), Carlos Rodríguez (Desarrollador Principal) y Elena Fernández (Diseñadora UX). El presupuesto asignado es de $25,000 USD."

**Esquema requerido:**
```json
{
  "proyecto": {"nombre":"string","presupuesto":{"monto":"number","moneda":"string"}},
  "reunion": {"fecha":"YYYY-MM-DD","hora":"HH:MM","ubicacion":"string"},
  "asistentes": [{"nombre":"string","rol":"string"}]
}
```

Se pide salida estricta con tipos y normalizaciones.


In [ ]:
# Código para extraer y normalizar la información del texto
import re, datetime, json, pprint

texto = ("La reunión de lanzamiento del Proyecto Fénix será el 15 de octubre de 2024 a las 10:30 AM "
         "en la Sala Júpiter. Los asistentes confirmados son Ana García (Líder de Proyecto), "
         "Carlos Rodríguez (Desarrollador Principal) y Elena Fernández (Diseñadora UX). "
         "El presupuesto asignado es de $25,000 USD.")

# Mapas y búsquedas
meses = {'enero':1,'febrero':2,'marzo':3,'abril':4,'mayo':5,'junio':6,'julio':7,'agosto':8,'septiembre':9,'octubre':10,'noviembre':11,'diciembre':12}

m_fecha = re.search(r'(\d{1,2}) de (\w+) de (\d{4})', texto)
fecha_iso = None
if m_fecha:
    dia = int(m_fecha.group(1))
    mes_str = m_fecha.group(2).lower()
    anio = int(m_fecha.group(3))
    mes = meses.get(mes_str)
    if mes:
        fecha_iso = f"{anio:04d}-{mes:02d}-{dia:02d}"

m_hora = re.search(r'([0-2]?\d:[0-5]\d)\s*(AM|PM)?', texto, re.IGNORECASE)
hora_24 = None
if m_hora:
    hora_raw = m_hora.group(1)
    ampm = (m_hora.group(2) or '').upper()
    dt = datetime.datetime.strptime(hora_raw, '%H:%M')
    if ampm == 'PM' and dt.hour < 12:
        dt = dt + datetime.timedelta(hours=12)
    if ampm == 'AM' and dt.hour == 12:
        dt = dt - datetime.timedelta(hours=12)
    hora_24 = dt.strftime('%H:%M')

m_ubic = re.search(r'en la ([^\.]+)\.', texto)
ubicacion = m_ubic.group(1).strip() if m_ubic else None

m_asist = re.search(r'Los asistentes confirmados son (.+?)\. El presupuesto', texto)
asistentes = []
if m_asist:
    lista = m_asist.group(1)
    partes = re.split(r',\s*', lista)
    # manejar ' y ' en el último
    if len(partes) > 0 and ' y ' in partes[-1]:
        last = partes[-1]
        antes, despues = last.rsplit(' y ', 1)
        partes[-1] = antes
        partes.append(después)
    asistentes = []
    for p in partes:
        p = p.strip()
        m = re.match(r'([^\(]+)\s*\(([^\)]+)\)', p)
        if m:
            nombre = m.group(1).strip()
            rol = m.group(2).strip()
        else:
            nombre = p
            rol = None
        asistentes.append({'nombre': nombre, 'rol': rol})

m_pres = re.search(r'\$([\d,]+)\s*([A-Za-z]{3})?', texto)
presupuesto = {'monto': None, 'moneda': None}
if m_pres:
    monto_str = m_pres.group(1)
    moneda = m_pres.group(2) or 'USD'
    monto = int(monto_str.replace(',', ''))
    presupuesto = {'monto': monto, 'moneda': moneda}

salida = {
    "proyecto": {"nombre": "Proyecto Fénix", "presupuesto": presupuesto},
    "reunion": {"fecha": fecha_iso, "hora": hora_24, "ubicacion": ubicacion},
    "asistentes": asistentes
}

pprint.pprint(salida)
print('\nJSON final:')
print(json.dumps(salida, ensure_ascii=False, indent=2))


**Respuesta simulada del modelo (prompt avanzado):**

```json
{
  "proyecto": {"nombre": "Proyecto Fénix", "presupuesto": {"monto": 25000, "moneda": "USD"}},
  "reunion": {"fecha": "2024-10-15", "hora": "10:30", "ubicacion": "Sala Júpiter"},
  "asistentes": [
    {"nombre": "Ana García", "rol": "Líder de Proyecto"},
    {"nombre": "Carlos Rodríguez", "rol": "Desarrollador Principal"},
    {"nombre": "Elena Fernández", "rol": "Diseñadora UX"}
  ]
}
```

**Reflexión:** El prompt avanzado dirige a producir JSON válido y parseable; la normalización de fecha/hora y el tipo numérico del presupuesto son esenciales.

---


### Ejercicio 3: Escritura creativa con tono y estilo

**Prompt Avanzado**: Rol: divulgador tipo Carl Sagan; Audiencia: niños 10-12; ~100 palabras; sin la palabra 'singularidad' ni 'relatividad general'.

**Respuesta simulada del modelo (aprox. 100 palabras):**


Los agujeros negros son como gigantescos aspiradores cósmicos: lugares en el espacio donde la gravedad es tan intensa que incluso la luz no puede escapar. Imagina una tela elástica extendida; si pones una bola pesada encima, la tela se hunde. Un agujero negro hunde el espacio a su alrededor dejando una depresión profunda. Los científicos los descubren observando cómo afectan a las estrellas y al material que cae en ellos, que brilla antes de desaparecer. Aunque suene misterioso, estudiar agujeros negros nos ayuda a entender cómo funciona el universo y lo que sucede cuando las cosas son extraordinariamente compactas.


**Reflexión:**
- El prompt avanzado logró un texto adecuado para niños: metáforas sencillas y tono de asombro.

---


## Parte 2 — EJERCICIOSPARTE2.md (Respuestas simuladas)

Incluye ejercicios: roles (historiador, humorista), plantillas, CoT, ReAct, Self-Consistency, ToT y JSON enforced output.


### Ejercicio 1B — Cambiando el rol a un Historiador

**Prompt:**
```
[system] Eres un historiador especializado en la Antigua Grecia. Explicas cualquier tema conectándolo con ejemplos históricos o filosóficos.
[user] ¿Qué es una variable?
```

**Respuesta simulada (historiador):**


Una variable, vista desde una lente histórica, sería comparable a los símbolos que los antiguos matemáticos griegos usaban para representar cantidades desconocidas en sus problemas —aunque ellos no contaban con la notación algebraica moderna. Imagina las incógnitas en los problemas que resolvían los pitagóricos: una variable es ese lugar reservado para lo que cambia o lo que aún no conocemos. Filosóficamente, recordaría a los conceptos de Heráclito sobre el cambio: la variable captura la idea de que algo puede variar dentro de una estructura fija, y permite razonar sobre relaciones antes de conocer valores concretos.


**Reflexión:**
- El historiador conecta el concepto con ejemplos antiguos y perspectivas filosóficas, útil para audiencias humanísticas.

---


### Ejercicio 1C — El Humorista

**Prompt:**
```
[system] Eres un comediante de stand-up. Explicas conceptos técnicos de manera graciosa.
[user] ¿Qué es una variable?
```

**Respuesta simulada (humorista):**


Una variable es como esa caja donde metes cosas cuando no quieres decidir qué hacer con ellas: "Puede ser un número, puede ser una cadena, ¡incluso puede ser tu ex!" En programación, le pones un nombre a la caja para que luego no tengas que abrirla con sorpresa cada vez. Si la variable fuera una persona, sería ese amigo indeciso que hoy es 5 y mañana es 'hola'. Muy versátil, poco confiable en fiestas.


**Reflexión:**
- El tono humorístico facilita la memorización, aunque sacrifica formalidad técnica.

---


### Sección 2: Plantillas Reutilizables

#### Ejemplo: Plantilla para resumir artículos científicos

**Prompt-plantilla (usada):**
```
# Identidad
Eres un investigador que resume artículos científicos para estudiantes universitarios.

# Instrucciones
- Haz un resumen de máximo 150 palabras.
- Usa un lenguaje accesible pero formal.
- Incluye: objetivo, método y conclusión principal.

# Contexto del Artículo
<articulo>
    <titulo>Aplicaciones de la IA en medicina</titulo>
    <autor>Dra. López</autor>
    <tema>Diagnóstico asistido por IA</tema>
</articulo>
```

**Respuesta simulada (resumen ≤150 palabras):**


La Dra. López revisa aplicaciones de la inteligencia artificial en diagnóstico médico, enfocándose en sistemas que apoyan la detección temprana de enfermedades. El objetivo es evaluar qué técnicas muestran mayor precisión en imágenes médicas. El método combina revisión sistemática de estudios clínicos y comparativas de rendimiento entre modelos de aprendizaje supervisado. Como conclusión, la autora encuentra que modelos entrenados con datos cuidadosamente anotados mejoran la detección precoz, pero advierte sobre sesgos en conjuntos de datos y la necesidad de evaluación clínica rigurosa antes de su despliegue.


**Plantilla para Emails Profesionales (ejemplo de uso):**

**Prompt:** se generó un email corto dirigido a un coordinador solicitando reunión. **Respuesta simulada:**

> Asunto: Solicitud de reunión para revisión de avance
>
> Estimado Juan,
>
> Espero que te encuentres bien. Quisiera coordinar una reunión de 30 minutos para revisar el avance del proyecto X y definir próximos pasos. ¿Puedes indicarme tus disponibilidades esta semana? Adjunto una breve agenda.
>
> Saludos,
> María

---


### Sección 3: Técnicas Avanzadas

#### 3.1 Chain-of-Thought (CoT)

**Ejemplo:** Calcular raíz cuadrada de 1764.

**Prompt con CoT solicitado:** "Calcula la raíz cuadrada de 1764. Piensa paso a paso, muestra tu razonamiento y luego la respuesta final."

**Respuesta simulada (pasos):**


1. Busco un número que multiplicado por sí mismo dé 1764.
2. Pruebo 40 → 40×40 = 1600 (demasiado bajo).
3. Pruebo 42 → 42×42 = 1764 (coincide).

**Respuesta final:** 42

(CoT ayudó a mostrar los pasos de verificación.)


#### 3.2 ReAct (ejemplo encadenado)

**Pregunta:** ¿Cuál es la capital de Australia y en qué continente está?

**Prompt con ReAct (simulado):**

```
Thought: Necesito identificar la capital de Australia.
Action: Search("capital de Australia")
Observation: Canberra es la capital.

Thought: Ahora debo identificar en qué continente está.
Action: Search("Australia continente")
Observation: Australia está en Oceanía.

Final Answer: La capital de Australia es Canberra y el país se encuentra en Oceanía.
```

**Nota:** Aquí simulamos acciones de búsqueda y observaciones para ilustrar la técnica.

---


#### 3.3 Self-Consistency — problema de probabilidad

**Problema:** Si lanzas una moneda dos veces, ¿cuál es la probabilidad de obtener dos caras seguidas?

**Discusión y respuesta correcta con razonamiento:**


Espacio muestral: {HH, HT, TH, TT} — 4 resultados iguales de probabilidad. Solo uno de ellos (HH) tiene dos caras. Por tanto, P = 1/4 = 0.25.

**Nota sobre self-consistency:** pedir varias cadenas de razonamiento y verificar la congruencia ayuda a evitar errores (p. ej. confundir con 1/2).

---


#### 3.4 Tree-of-Thought (ToT) — Planificación de viaje (ejemplo)

**Prompt:** "Explora 3 posibles planes de viaje a Europa con presupuesto limitado. Paso 1: genera 3 opciones. Paso 2: evalúa costo, cultura y accesibilidad. Paso 3: elige la mejor y explica por qué."

**Respuesta simulada (ToT):**


**Opciones generadas:**

1. **España (Madrid + ciudades cercanas)**
   - Costo: Moderado (vuelos desde Sudamérica pueden ser más caros, pero transporte interno barato si se usa tren/autobús).
   - Cultura: Alta (museos, arquitectura, gastronomía).
   - Accesibilidad: Buena (aeropuertos internacionales, conexiones de tren).

2. **Portugal (Lisboa + Oporto)**
   - Costo: Bajo-moderado (generalmente más barato que España en alojamiento y comida).
   - Cultura: Rica (historia marítima, barrios tradicionales).
   - Accesibilidad: Buena, vuelos directos y transporte local económico.

3. **Hungría (Budapest)**
   - Costo: Bajo (alojamiento y comidas económicas en Europa central).
   - Cultura: Buena (baños termales, patrimonio arquitectónico).
   - Accesibilidad: Media (vuelos directos limitados, podría requerir escala).

**Elección recomendada:** Portugal por su balance costo/experiencia y facilidad de transporte local; si buscas mayor vida nocturna y museos, España es la alternativa.

---


#### 3.5 JSON Enforced Output (Extracción de contactos)

**Texto de entrada:** "Juan Pérez vive en Bogotá, Colombia. Su correo es juanperez@mail.com y su teléfono es +57 300 123 4567."

**Prompt:** Extrae la información y responde solo con JSON:
```json
{
  "nombre":"string",
  "ciudad":"string",
  "pais":"string",
  "email":"string",
  "telefono":"string"
}
```

**Respuesta simulada (JSON):**


```json
{
  "nombre": "Juan Pérez",
  "ciudad": "Bogotá",
  "pais": "Colombia",
  "email": "juanperez@mail.com",
  "telefono": "+57 300 123 4567"
}
```

**Reflexión:** Forzar salida JSON reduce errores de parseo cuando se integra con pipelines automáticos.

---


## Cierre y recomendaciones finales

- Los prompts avanzados (rol, formato de salida, restricciones, ejemplos) producen resultados más consistentes y útiles.
- Técnicas como CoT, ReAct, Self-Consistency y ToT ayudan en distintos tipos de razonamiento: pasos, búsqueda externa simulada, verificación y exploración de alternativas respectivamente.

---
